In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from tqdm import trange

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 192 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 4
n_layer = 8
dropout = 0.1
# ------------

torch.manual_seed(1337)

with open('../data/shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
chars.append('PAD')
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

6.395458 M parameters


In [3]:
model.state_dict()

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.2103, -0.1126, -0.3808,  ..., -2.0068, -0.5345,  1.2165],
                      [ 0.3902, -1.0554, -0.7819,  ...,  0.0449, -0.3861, -0.4618],
                      [-0.6071,  1.9650,  0.1719,  ...,  0.6113, -0.9235,  0.6474],
                      ...,
                      [ 0.4150,  0.3674, -0.3707,  ...,  0.5631,  0.3634,  0.5290],
                      [-2.2017,  0.4827,  0.7358,  ..., -0.4392,  2.4382,  1.9693],
                      [-0.4847,  0.1756, -0.7382,  ..., -0.7080, -0.2190, -0.1110]],
                     device='cuda:0')),
             ('position_embedding_table.weight',
              tensor([[-0.3906, -1.6061, -1.0013,  ...,  1.2565,  0.2262, -1.1616],
                      [-0.7695,  0.0297, -0.1585,  ..., -1.3447, -2.0665, -0.5862],
                      [-0.9585, -0.3573, -0.5156,  ..., -2.1883, -0.2799,  1.7498],
                      ...,
                      [-0.3948, -1.1632, -0.2605,  ...

In [2]:
checkpoint = torch.load('./saved_models/trained_shakespeare/trained_shakespeare.pth', map_location='cpu')

# Load the state dict from the checkpoint
state_dict = checkpoint['model_state_dict']


# Update the state dict of the sentiment model
model.load_state_dict(state_dict, strict=True)

model = model.to(device)

In [3]:
max_iters = 1000
for iter in trange(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|          | 3/1000 [00:06<1:09:50,  4.20s/it]

step 0: train loss 1.3414, val loss 1.5588


 10%|█         | 103/1000 [00:17<09:36,  1.55it/s]

step 100: train loss 1.3402, val loss 1.5548


 20%|██        | 203/1000 [00:28<08:51,  1.50it/s]

...

step 800: train loss 1.2807, val loss 1.5376


 90%|█████████ | 903/1000 [01:48<01:07,  1.43it/s]

step 900: train loss 1.2720, val loss 1.5233


100%|██████████| 1000/1000 [01:59<00:00,  8.37it/s]

step 999: train loss 1.2699, val loss 1.5315


In [7]:
model.state_dict()

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.1769, -0.0410, -0.3897,  ..., -1.9596, -0.5568,  1.0564],
                      [ 0.4193, -1.0834, -0.7988,  ...,  0.0537, -0.3840, -0.5005],
                      [-0.5921,  1.9393,  0.3036,  ...,  0.4922, -0.9253,  0.6576],
                      ...,
                      [ 0.3351,  0.3965, -0.3717,  ...,  0.4652,  0.3239,  0.5393],
                      [-2.1822,  0.6954,  0.6643,  ..., -0.1460,  2.3360,  2.0221],
                      [-0.4871,  0.1765, -0.7419,  ..., -0.7115, -0.2201, -0.1116]],
                     device='cuda:0')),
             ('position_embedding_table.weight',
              tensor([[-0.2303, -1.5577, -0.9424,  ...,  1.2698,  0.2820, -1.1742],
                      [-0.7510,  0.0427, -0.1967,  ..., -1.2779, -1.9479, -0.5975],
                      [-0.8943, -0.3436, -0.4935,  ..., -2.1485, -0.2589,  1.7064],
                      ...,
                      [ 1.2827,  0.1181,  0.2124,  ...

In [4]:
torch.save({
    'model_state_dict': model.state_dict(), 
    'optimizer_state_dict': optimizer.state_dict()
    }, './saved_models/trained_shakespeare/trained_shakespeare.pth')

In [7]:
checkpoint = torch.load('./saved_models/trained_shakespeare/trained_shakespeare.pth', map_location='cpu')

In [8]:
state_dict = checkpoint['model_state_dict']
state_dict

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.1769, -0.0410, -0.3897,  ..., -1.9596, -0.5568,  1.0564],
                      [ 0.4193, -1.0834, -0.7988,  ...,  0.0537, -0.3840, -0.5005],
                      [-0.5921,  1.9393,  0.3036,  ...,  0.4922, -0.9253,  0.6576],
                      ...,
                      [ 0.3351,  0.3965, -0.3717,  ...,  0.4652,  0.3239,  0.5393],
                      [-2.1822,  0.6954,  0.6643,  ..., -0.1460,  2.3360,  2.0221],
                      [-0.4871,  0.1765, -0.7419,  ..., -0.7115, -0.2201, -0.1116]])),
             ('position_embedding_table.weight',
              tensor([[-0.2303, -1.5577, -0.9424,  ...,  1.2698,  0.2820, -1.1742],
                      [-0.7510,  0.0427, -0.1967,  ..., -1.2779, -1.9479, -0.5975],
                      [-0.8943, -0.3436, -0.4935,  ..., -2.1485, -0.2589,  1.7064],
                      ...,
                      [ 1.2827,  0.1181,  0.2124,  ...,  0.2524,  1.6898, -0.0875],
        

In [4]:
# generate from the model
context = torch.tensor(encode('CLARK:'), device=device).view(1, -1)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

CLARK:
I was for time and Richard Hearts;
'Tis trusting like a that enarm know now the world,
And barrack daw-fareweak socrow:
Our will, have death fourth, and the landably, or us,
From marriageness of the kaver, shall any give enorat.

JULIET:
O will fellower in ourstome woo'ds, Captol-Puriom!

FinIONTES:
We our will apprabed I clow answer, and love CAMILLO:
What's thee at was to live?

CAPULET:
He as go. A high 'gone, deter!'

Shepherd:
I go repent.

Citizen:
No, ir, if your suiers
Beseeming-hors
Up


In [4]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


And they bride will that yet that set beast in an grave
My have us bard that us an her back that a enfrice,
And folly unzurn heavens, to that I covert;
And that misters, acheils is there by on,
Which news is wan thelpsing of his hunt couries:
Suppiniss hiw yet longer nor does it down.

GLOUCESTER:
It on, not what evil so, and do rive cheeks,
As if the enforgund kindness firf so;
And he must with all on, that Prison, Some,
Her such stild is a adday the Earl of dights.

DUKE VINCENTIO:

QUKEENEN E
